# Integrating MCP Servers to an OpenAI Agent
Introduction & Lesson Overview

Welcome! Now that you’ve built your MCP server and exposed your shopping list tools, it’s time to make them available to an OpenAI agent. In this lesson, you’ll learn how to connect your MCP server to an agent using both local (stdio) and remote (SSE) transports, how to provide the server to the agent, and how to test that the integration works as expected.

By the end of this lesson, you’ll be able to:

    Connect an OpenAI agent to your MCP server using both stdio and SSE transports.
    Provide the MCP server to the agent so it can discover and use your tools.
    Run and test the integration, verifying that the agent can answer queries using your shopping list service.

Let’s walk through each step in detail.
Connecting the Agents SDK to an MCP Server via Stdio

The simplest way to connect your MCP server to an agent is by using the stdio transport. This is ideal for local development, where your server runs as a subprocess on the same machine as your agent. Communication happens over standard input and output, making it fast and easy to set up.

Here’s how you can launch your MCP server and connect to it via stdio using the OpenAI Agents SDK:

Python

import asyncio

from agents.mcp import MCPServerStdio


async def main():

    # Define how to start your MCP server as a subprocess

    server_params = {

        "command": "python",

        "args": ["mcp_server.py"]

    }


    # Create a connection to the MCP server using stdio

    async with MCPServerStdio(params=server_params) as mcp_server:

        # Your agent code goes here...


if __name__ == "__main__":

    asyncio.run(main())

    The command and args specify how to launch your MCP server script.
    The MCPServerStdio context manager handles starting and stopping the server process for you.

This setup is perfect for development and testing on your own machine.
Connecting the Agents SDK to an MCP Server via SSE

If your MCP server is running remotely—perhaps on another machine or in the cloud—you’ll want to use the SSE (Server-Sent Events) transport. This allows the agent to communicate with your server over HTTP, making it suitable for distributed or production environments.

Here’s how to connect using SSE:

Python

import asyncio

from agents.mcp import MCPServerSse


async def main():

    # Specify the URL where your MCP server is running

    server_params = {"url": "http://localhost:3000/sse"}


    # Create a connection to the MCP server using SSE

    async with MCPServerSse(params=server_params) as mcp_server:

        # Your agent code goes here...


if __name__ == "__main__":

    asyncio.run(main())

    Replace the URL with the address of your running MCP server.
    The MCPServerSse context manager manages the HTTP connection for you.

This approach is great for connecting to servers that are not running on your local machine.
Providing the MCP Server to the Agent

Once you have an mcp_server object—whether from MCPServerStdio or MCPServerSse—you can provide it to your agent. The agent will automatically discover all the tools your server exposes, read their documentation and input schemas, and use them to answer user queries.

Here’s a complete example using the stdio transport:

Python

import asyncio

from agents import Agent, Runner

from agents.mcp import MCPServerStdio


async def main():

    server_params = {

        "command": "python",

        "args": ["mcp_server.py"]

    }


    async with MCPServerStdio(params=server_params) as mcp_server:

        # Create the agent and provide the MCP server

        agent = Agent(

            name="Shopping Assistant",

            instructions="You are an assistant that uses shopping list tools to help manage a shopping list.",

            model="gpt-4.1",

            mcp_servers=[mcp_server]  # List of MCP servers available

        )


        # Run the agent with a query

        result = await Runner.run(agent, "Give me my shopping list")

        

        # Print the final output

        print(result.final_output)


if __name__ == "__main__":

    asyncio.run(main())

    The mcp_servers argument is a list, so you can provide one or more MCP server connections.
    The agent will aggregate all available tools from the connected servers.
    When you run the script, the agent connects to your MCP server, discovers the tools, and uses them to answer the query.

You can use the same approach with MCPServerSse by swapping out the context manager.
How the Agent Discovers and Uses Your Tools

When you provide the MCP server to the agent, the agent automatically connects and fetches all available tools. It reads the documentation and input schemas you defined with the @mcp.tool() decorator. This means the agent knows what each tool does and how to use it—no extra programming required.

For example:

    If you ask, “Give me my shopping list”, the agent will recognize it can use the fetch_items tool.
    If you say, “Add 3 bananas to my shopping list”, the agent will use the add_item tool with the correct parameters.

This automatic discovery and aggregation of tools is what makes MCP integration so powerful. The agent can flexibly use any tool you expose, based on the user’s request.
Lesson Summary & Next Steps

In this lesson, you learned how to connect an OpenAI agent to your MCP server using both stdio and SSE transports. You saw how to provide the MCP server to the agent, allowing it to automatically discover and use your shopping list tools in response to natural language queries. You also learned how to run and test the integration, verifying that your tools are accessible to the agent.

You’re now ready to practice these skills by building and testing your own agent-server integrations. This is a major step forward—your tools are now available to intelligent agents that can use them in flexible, conversational ways. In the next exercises, you’ll get hands-on experience with these integrations and deepen your understanding even further.


In [ ]:
# Connecting Agents SDK to MCP Server via stdio

import asyncio
from agents.mcp import MCPServerStdio

async def main():
    # Define how to start your MCP server as a subprocess
    server_params = {
        "command": "python",
        "args": ["mcp_server.py"]
    }

    # Create a connection to the MCP server using stdio
    async with MCPServerStdio(params=server_params) as mcp_server:
        # Your agent code goes here...

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
# Connecting Agents SDK to MCP Server via SSE

import asyncio
from agents.mcp import MCPServerSse

async def main():
    # Specify the URL where your MCP server is running
    server_params = {"url": "http://localhost:3000/sse"}

    # Create a connection to the MCP server using SSE
    async with MCPServerSse(params=server_params) as mcp_server:
        # Your agent code goes here...

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
# MCP Server to AI Agent via stdio

import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerStdio

async def main():
    server_params = {
        "command": "python",
        "args": ["mcp_server.py"]
    }

    async with MCPServerStdio(params=server_params) as mcp_server:
        # Create the agent and provide the MCP server
        agent = Agent(
            name="Shopping Assistant",
            instructions="You are an assistant that uses shopping list tools to help manage a shopping list.",
            model="gpt-4.1",
            mcp_servers=[mcp_server]  # List of MCP servers available
        )

        # Run the agent with a query
        result = await Runner.run(agent, "Give me my shopping list")
        
        # Print the final output
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

# Lesson 1

It’s finally time to put MCP and Agents together! Your first task is to modify the provided main.py so that our OpenAI agent connects to the shopping list MCP server using stdio. To do this, you will:

    Import the correct class for stdio-based MCP server connections from the OpenAI Agents SDK.
    Define the server parameters so the agent knows how to launch the MCP server as a subprocess.
    Use an async context manager to handle the MCP server connection.
    Make sure the agent is created with the connected MCP server in its mcp_servers list.

Once you’ve made these changes, the agent should be able to run a query like “Give me my shopping list” and get a response using the tools from your MCP server. Give it a try and see your integration in action!

In [ ]:
import asyncio
from agents import Agent, Runner
# TODO: Import MCPServerStdio from agents.mcp
from agents.mcp import MCPServerStdio


async def main():
    # TODO: Define the server parameters (command and args for mcp_server.py)
    server_params = {
        "command": "python",
        "args": ["mcp_server.py"]
    }

    # TODO: Use an async with statement to connect to the MCP server via stdio and assign the connection to a variable
    async with MCPServerStdio(params=server_params) as mcp_server:
        
    # TODO: Inside the context, create the agent and provide the MCP server
        agent = Agent(
            name="Shopping Assistant",
            instructions="You are an assistant that uses shopping list tools to help manage a shopping list.",
            model="gpt-4.1",
            # TODO: Add the mcp_servers parameter with the connected MCP server
            mcp_servers=[mcp_server]
        )

    # Run the agent with the single query
    result = await Runner.run(
        starting_agent=agent,
        input="Give me my shopping list"
    )

    # Print the final output
    print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

# Lesson 2

You’ve already connected your agent to the MCP server using the local stdio transport. Now, let’s take the next step and switch your integration to use the SSE transport, which is perfect for connecting to a server running remotely or on another port.

The MCP server is already set up and running via SSE on port 3000. Your task is to:

    Change the import in your main.py so that it uses MCPServerSse instead of MCPServerStdio.
    Update the server parameters to use a URL, such as "http://localhost:3000/sse", instead of the command and arguments.
    Adjust the context manager so that it uses MCPServerSse.

Once you’ve made these changes, your agent should be able to connect to the MCP server over HTTP and still answer queries like “Give me my shopping list” using the shopping list tools.

This will help you see how easy it is to switch between local and remote transports for your agent-server integration.

In [ ]:
import asyncio
from agents import Agent, Runner
# TODO: Change the import to MCPServerSse from agents.mcp
from agents.mcp import MCPServerSse


async def main():
    # Specify the URL where your MCP server is running
    server_params = {"url": "http://localhost:3000/sse"}

    # Create a connection to the MCP server using SSE
    async with MCPServerSse(params=server_params) as mcp_server:
        # Create the agent and provide the MCP server
        agent = Agent(
            name="Shopping Assistant",
            instructions="You are an assistant that uses shopping list tools to help manage a shopping list.",
            model="gpt-4.1",
            mcp_servers=[mcp_server]
        )

        # Run the agent with the single query
        result = await Runner.run(
            starting_agent=agent,
            input="Give me my shopping list"
        )

        # Print the final output
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

# Exercise 3

You’ve already seen how to connect your agent to the MCP server and have tried out some simple queries. Now, let’s see how well your agent handles a more involved, multi-step request.

Your task is to update the query so the agent is given a complex prompt that requires it to:

    Figure out all the ingredients needed to make chocolate chip cookies.
    Add each of those ingredients to your shopping list using the available tools.
    Retrieve and display the full shopping list, showing which items have been purchased and which have not.

This is a great way to check whether your agent can break down a bigger task, use the right tools, and give a complete answer. Give it a try and see how your agent performs!

In [ ]:
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerStdio


async def main():
    # Set up the MCP server parameters for stdio
    server_params = {
        "command": "python",
        "args": ["mcp_server.py"]
    }

    # Connect to the MCP server via stdio
    async with MCPServerStdio(params=server_params) as mcp_server:
        # Create an agent with basic instructions
        agent = Agent(
            name="Shopping Assistant",
            instructions="You are an assistant that uses shopping list tools to help manage a shopping list.",
            mcp_servers=[mcp_server],
            model="gpt-4.1"
        )

        # TODO: Replace the simple query below with a complex, multi-step query
        result = await Runner.run(
            starting_agent=agent,
            input="I would like you to figure out all the ingredients needed to make chocolate chip cookies. Add each of those ingredients to the shopping list using the available tools. Retrieve and display the full shopping list, showing which items have been purchased and which have not."
        )

        # Print the final output
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

Here is your current shopping list, including all the ingredients needed to make chocolate chip cookies. Items needed for cookies are marked with an asterisk (*):

Purchased:
- Milk (2)
- Eggs (12)

Not purchased:
- Bread (1)
- Apples (6)
- Coffee (1)

For chocolate chip cookies:
- All-purpose flour* (1) – Not purchased
- Baking soda* (1) – Not purchased
- Salt* (1) – Not purchased
- Unsalted butter* (1) – Not purchased
- Granulated sugar* (1) – Not purchased
- Brown sugar* (1) – Not purchased
- Vanilla extract* (1) – Not purchased
- Eggs* (2) – Not purchased
- Chocolate chips* (1) – Not purchased

Would you like me to mark cookie ingredients in any special way or remove items not related to cookies?

# Final exercise

Now, let’s take it a step further and create a multi-turn conversation in which the agent keeps track of what has happened so far.

Instead of running a single query, you'll iterate through a list of prompts, and after each agent response, you'll construct the next input by combining the previous conversation history (using result.to_input_list()) with the new user prompt (as a dictionary: {"role": "user", "content": prompt}). This way, each call to the agent includes the full context of the conversation so far, allowing the agent to maintain state and respond accurately across multiple turns.

Be sure to use print statements to show the output after each step so you can see how the conversation and the shopping list change. This exercise will help you see how to chain agent interactions and maintain context across multiple turns.

In [ ]:
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerStdio


async def main():
    # Set up the MCP server parameters for stdio
    server_params = {
        "command": "python",
        "args": ["mcp_server.py"]
    }

    # Connect to the MCP server via stdio
    async with MCPServerStdio(params=server_params) as mcp_server:
        # Create an agent with clear instructions
        agent = Agent(
            name="Shopping Assistant",
            instructions="You are an assistant that uses shopping list tools to help manage a shopping list.",
            mcp_servers=[mcp_server],
            model="gpt-4.1"
        )

        # Define the prompts for each step of the conversation
        prompts = [
            "List all items in my shopping list",
            "Mark 'Coffee' as purchased in my shopping list",
            "List all purchased items in my shopping list"
        ]

        # Holds the result of the previous step in the conversation
        result = None

        # TODO: Loop through each prompt, chaining the conversation
        for prompt in prompts:
        # For each prompt:
        #   - If this is the first turn, run the agent with just the prompt
        #   - Otherwise, use result.to_input_list() + [{"role": "user", "content": prompt}] as input
        #   - Print the output for each step
            if result == None:
                result = await Runner.run(agent, prompt)
                
            else:
                conversation_history = result.to_input_list()
                conversation_history += [{"role": "user", "content": prompt}]
                result = await Runner.run(agent, conversation_history)
            print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())